In [1]:
import sympy as sp
import numpy as np
from sympy import Symbol, hessian, simplify, count_ops
from functools import reduce
from sympy.utilities import codegen
from sympy.codegen.rewriting import optimize, optims_c99, create_expand_pow_optimization
import os

def jacobian_for(expr, vars):
    return sp.Matrix([expr]).jacobian(vars)

def hessian_for(expr, vars):
    return hessian(expr, vars)

def generate_code_for(expr, vars, name):
    oe = optimize(expr, optims_c99)
    cc = codegen.C99CodeGen(project = "acg-3rdparty-ipc", cse=True)
    Eq = sp.Eq
    [(cn, cc), (hn, hc)] = codegen.codegen(
        [(name + "_value", oe),
         (name + "_grad", jacobian_for(oe, vars)),
         (name + "_hessian", hessian_for(oe, vars))], code_gen=cc)
    if not os.path.exists('generated'):
        os.mkdir('generated')
    with open("generated/" + cn, mode='w') as f:
        f.write(cc)
    with open("generated/" + hn, mode='w') as f:
        f.write(hc)



# Definition of Barrier Function

Barrier is defined as:

$$
b(d, \hat d) = \begin{cases}
-(d - \hat d)^2 \log (d/\hat d)& d < \hat d,\\
0, & otherwise
\end{cases}
$$

In [2]:
dhat = sp.var('d_h')
def dist_barrier(d):
    return - (d - dhat) * (d - dhat) * sp.ln(d / dhat)

## test

In [3]:
dist_barrier(Symbol('d'))

(-d + d_h)*(d - d_h)*log(d/d_h)

# Vertex-Face Barrier

- Face = $a, b, c$
- Vertex = $d$

In [4]:
def declare_vector(name, dim):
    return sp.Matrix(sp.symbols([f'{name}_{i}' for i in range(dim)], real=True))

a = declare_vector('a', 3)
b = declare_vector('b', 3)
c = declare_vector('c', 3)
d = declare_vector('d', 3)
free_symbols = reduce(lambda x, y: x + y, [a.tolist(), b.tolist(), c.tolist(), d.tolist()])
free_symbols = reduce(lambda x, y: x + y, free_symbols)
free_symbols

[a_0, a_1, a_2, b_0, b_1, b_2, c_0, c_1, c_2, d_0, d_1, d_2]

## Normal of the triangle

In [5]:
normal = (b - a).cross(c - a)
normal_normal = normal.normalized()
normal_normal[0]

((-a_1 + b_1)*(-a_2 + c_2) - (-a_1 + c_1)*(-a_2 + b_2))/sqrt(((-a_0 + b_0)*(-a_1 + c_1) - (-a_0 + c_0)*(-a_1 + b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((-a_1 + b_1)*(-a_2 + c_2) - (-a_1 + c_1)*(-a_2 + b_2))**2)

## Offset from triangle vertex 0 to extra vertex

In [6]:
off = (d - a)
off

Matrix([
[-a_0 + d_0],
[-a_1 + d_1],
[-a_2 + d_2]])

## Distance and barrier

In [7]:
distance = simplify(normal_normal.dot(off))
distance2 = simplify(distance * distance)
distance2

((a_0 - d_0)*((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2)) - (a_1 - d_1)*((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2)) + (a_2 - d_2)*((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1)))**2/(((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2)

In [8]:
barrier = dist_barrier(distance2)
# TODO: Simplify the barrier cost a lot of time.
barrier

(-d_h + ((a_0 - d_0)*((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2)) - (a_1 - d_1)*((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2)) + (a_2 - d_2)*((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1)))**2/(((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2))*(d_h - ((a_0 - d_0)*((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2)) - (a_1 - d_1)*((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2)) + (a_2 - d_2)*((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1)))**2/(((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2))*log(((a_0 - d_0)*((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2)) - (a_1 - d_1)*((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2)) + (a_2 - d_2)*((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1)))**2/(d_h*(((a_0 - b_0)*(a_1 - c_1) - (

## Hessian and jacobian for barrier

In [9]:
generate_code_for(barrier, free_symbols, "vertex_face")

# Distance for Edge and Edge

- Edge 0: $a, b$
- Edge 1: $c, d$

In [10]:
e0 = a - b
e1 = c - d
ee_norm = e0.cross(e1).normalized()
ee_norm

Matrix([
[ ((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1))/sqrt(((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0))**2 + ((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0))**2 + ((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1))**2)],
[(-(a_0 - b_0)*(c_2 - d_2) + (a_2 - b_2)*(c_0 - d_0))/sqrt(((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0))**2 + ((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0))**2 + ((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1))**2)],
[ ((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0))/sqrt(((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0))**2 + ((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0))**2 + ((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1))**2)]])

In [11]:
ee_distance = (c - a).dot(ee_norm)
ee_distance2 = simplify(ee_distance * ee_distance)
ee_distance2

((a_0 - c_0)*((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1)) - (a_1 - c_1)*((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0)) + (a_2 - c_2)*((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0)))**2/(((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0))**2 + ((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0))**2 + ((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1))**2)

In [12]:
ee_barrier = dist_barrier(ee_distance2)
ee_barrier

(-d_h + ((a_0 - c_0)*((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1)) - (a_1 - c_1)*((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0)) + (a_2 - c_2)*((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0)))**2/(((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0))**2 + ((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0))**2 + ((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1))**2))*(d_h - ((a_0 - c_0)*((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1)) - (a_1 - c_1)*((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0)) + (a_2 - c_2)*((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0)))**2/(((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0))**2 + ((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0))**2 + ((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1))**2))*log(((a_0 - c_0)*((a_1 - b_1)*(c_2 - d_2) - (a_2 - b_2)*(c_1 - d_1)) - (a_1 - c_1)*((a_0 - b_0)*(c_2 - d_2) - (a_2 - b_2)*(c_0 - d_0)) + (a_2 - c_2)*((a_0 - b_0)*(c_1 - d_1) - (a_1 - b_1)*(c_0 - d_0)))**2/(d_h*(((a_0 - b_0)*(c_1 - d_1) - (

In [13]:
generate_code_for(ee_barrier, free_symbols, 'edge_edge')

# Edge-Vertex Distance

- Edge: $a, b$
- Vertex: $c$

In [14]:
ev_distance = e0.normalized().cross(c - a).norm()
ev_distance2 = simplify(ev_distance * ev_distance)
ev_distance2

(((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2)/((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2)

In [15]:
ev_barrier = dist_barrier(ev_distance2)
ev_barrier

(-d_h + (((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2)/((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2))*(d_h - (((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2)/((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2))*log((((a_0 - b_0)*(a_1 - c_1) - (a_0 - c_0)*(a_1 - b_1))**2 + ((a_0 - b_0)*(a_2 - c_2) - (a_0 - c_0)*(a_2 - b_2))**2 + ((a_1 - b_1)*(a_2 - c_2) - (a_1 - c_1)*(a_2 - b_2))**2)/(d_h*((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2)))

In [16]:
ev_free = free_symbols[:9]
ev_free

[a_0, a_1, a_2, b_0, b_1, b_2, c_0, c_1, c_2]

In [17]:
generate_code_for(ev_barrier, ev_free, 'edge_vertex')

# Vertex-Vertex

V: $a, b$

In [18]:
vv_dist = simplify(e0.norm())
vv_dist

sqrt((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2)

In [19]:
vv_barrier = simplify(dist_barrier(vv_dist))
vv_barrier

-(d_h - sqrt((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2))**2*log(sqrt((a_0 - b_0)**2 + (a_1 - b_1)**2 + (a_2 - b_2)**2)/d_h)

In [20]:
generate_code_for(vv_barrier, free_symbols[:6], 'vertex_vertex')